In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3/4 - 3.6/4.4 Marcação de Aprovados

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.saidas.mensagemFinal':["99. Accepted"],
                             'payloadHomol.intermediarias.aprovadoBAUAux' :[1],
                             'payloadHomol.intermediarias.aprovadoSLAux':[0],
                             'payloadHomol.intermediarias.digitoCPFConcomitanteCP':[99],
                             'solicitante.flagSmallLimits':[0],
                            })
                            

In [0]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_MarcacoesAprovados = sherlock.Gera_Massa(base_entrada)
massa_MarcacoesAprovados.gera_cenario('payloadHomol.saidas.mensagemFinal',["97. PPCard Credito Contratado",
                                                                           "98. LG Contratado",
                                                                           "26. Score <= 454",
                                                                           "27. Score Invalido Concessao",
                                                                           "29. BlockList Concessao",
                                                                           "02. Cadastro Irregular",
                                                                           "03. Cadastro Negado ou Em Andamento",
                                                                           "04. Conta Inativa ou Conta Teste PicPay",
                                                                           "05. Sistema Operacional Defasado",
                                                                           "06. CPF Invalido",
                                                                           "07. Obito",
                                                                           "09. Fraude PicPay",
                                                                           "10. Chargeback PicPay",
                                                                           "11. Idade <= 20 ou >= 80",
                                                                           "12. Atraso PicPay",
                                                                           "13. Atraso Original",
                                                                           "14. Reneg ou FNV Ativo",
                                                                           "15. Valor de Restritivo BVS >= 10% da Renda",
                                                                           "16. Valor de Restritivo Serasa >= 10% da Renda",
                                                                           "17. Quantidade de IFs = 0",
                                                                           "18. Atraso Bacen >= 10% da Renda",
                                                                           "19. Prejuizo Bacen >= 10% da Renda",
                                                                           "20. Reneg Bacen",
                                                                           "21. Saldo Financiado de Cartao > 70% do Limite",
                                                                           "22. IU Cheque > 90%",
                                                                           "23. AD Bacen",
                                                                           "24. Desenrola",
                                                                           "25. No hit Bureaus",
                                                                           "99. Accepted"])
massa_MarcacoesAprovados.gera_cenario('payloadHomol.intermediarias.aprovadoBAUAux',[0,1])
massa_MarcacoesAprovados.gera_cenario('payloadHomol.intermediarias.aprovadoSLAux',[0,1])
massa_MarcacoesAprovados.gera_cenario('solicitante.flagSmallLimits',[0,1])
massa_MarcacoesAprovados.gera_cenario('payloadHomol.intermediarias.digitoCPFConcomitanteCP',[0,49,50,51,73,74,75,99])

massa_MarcacoesAprovados.set_bom_arqv()
massa_MarcacoesAprovados.converte_massa()

In [0]:
massa_MarcacoesAprovados.get_massa_final()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_MarcacoesAprovados.get_massa_json(),rp.MarcacoesAprovados)

##### Valida se todas condições estão sendo abordadas

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['flagAprovadoBAU'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['flagConcomitanteSLCP'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['flagAprovadoSL'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_MarcacoesAprovados.rename_cabecalho()

In [0]:
massa_MarcacoesAprovados.atribui_expected(listaResultado,"payloadHomol.intermediarias.flagAprovadoBAU","aprovadoBAU")
massa_MarcacoesAprovados.atribui_expected(listaResultado,"payloadHomol.intermediarias.flagAprovadoSL","aprovadoSmallLimits")
massa_MarcacoesAprovados.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_MarcacoesAprovados.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaMarcacoesAprovados.csv',index=False)